In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import seaborn as sns
import numpy as np


In [2]:
device=torch.device('cuda'if torch.cuda.is_available else 'cpu')

In [16]:
num_epochs=4
batch_size=4
lr=0.001

In [4]:
transform=transforms.Compose(
[transforms.ToTensor(),
transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
train=torchvision.datasets.CIFAR10(root='./content/data',train=True,download=True,transform=transform)
test=torchvision.datasets.CIFAR10(root='./content/data',train=False,download=True,transform=transform)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./content/data/cifar-10-python.tar.gz to ./content/data
Files already downloaded and verified


In [5]:
train_dataloader=torch.utils.data.DataLoader(train,batch_size=batch_size,shuffle=True)
test_dataloader=torch.utils.data.DataLoader(test,batch_size=batch_size,shuffle=False)

In [6]:
classes=('deer','dog','frog','horse','ship','truck','car','bird','cat','plane')

In [7]:
sample_image=iter(train_dataloader)

In [8]:
samples,labels=sample_image.next()
print(samples.shape) #64 batch size, 1 channel, width 28 , height 28
print(labels.shape)

torch.Size([16, 3, 32, 32])
torch.Size([16])


In [ ]:
# CONV LAYER FORMULA (W - F + 2P)/S+1
# W= width
# F= kernel_size
# P= padding
# S =strides

In [9]:
# REASON OF THIS CODE IS TO CHECK WHAT INPUTS TO PUSH IN DENSE LAYER
x1=nn.Conv2d(3,6,5)
pool=nn.MaxPool2d(2,2)
conv2=nn.Conv2d(6,14,5)

print(samples.shape)
x=x1(samples)
print(x.shape)

x=pool(x)
print(x.shape)
x=conv2(x)
print(x.shape)

x=pool(x)
print(x.shape)


torch.Size([16, 3, 32, 32])
torch.Size([16, 6, 28, 28])
torch.Size([16, 6, 14, 14])
torch.Size([16, 14, 10, 10])
torch.Size([16, 14, 5, 5])


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [27]:
class ConvolutionNet(nn.Module):
    def __init__(self):
        super(ConvolutionNet, self).__init__()
        self.conv1=nn.Conv2d(3,6,5) #input_channel 3 , output_channel is 6
        self.pool=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(6,16,5)
        self.fc1=nn.Linear(16*5*5,120) 
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
        
        
    def forward(self,x):
        x=self.conv1(x)
        x=F.relu(x)
        x=self.pool(x)
        x=self.conv2(x)
        x=F.relu(x)
        x=self.pool(x)
        x=x.view(-1,16*5*5)
        x= self.fc1(x)
        x=F.relu(x)
        x=self.fc2(x)
        x=F.relu(x)
        x=self.fc3(x)
        return x
    


In [29]:
model=ConvolutionNet().to(device)
criterion=nn.CrossEntropyLoss() #softmax already included
optimizer=torch.optim.SGD(model.parameters(),lr=lr)

In [30]:
model

ConvolutionNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [31]:

n_total_steps = len(train_dataloader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dataloader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

Epoch [1/4], Step [2000/3125], Loss: 2.2988
Epoch [2/4], Step [2000/3125], Loss: 2.2839
Epoch [3/4], Step [2000/3125], Loss: 2.1121
Epoch [4/4], Step [2000/3125], Loss: 1.9401
Finished Training


In [34]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 28.29 %
Accuracy of deer: 52.42290748898679 %
Accuracy of dog: 39.67611336032389 %
Accuracy of frog: 0.7194244604316546 %
Accuracy of horse: 13.46938775510204 %
Accuracy of ship: 0.7407407407407407 %
Accuracy of truck: 24.166666666666668 %
Accuracy of car: 67.10526315789474 %
Accuracy of bird: 19.37984496124031 %
Accuracy of cat: 27.125506072874494 %
Accuracy of plane: 34.23076923076923 %
